### Startup imports

In [ ]:
import pandas as pd

from autoqchem.molecule import molecule
from autoqchem.slurm_manager import lsf_manager
from autoqchem.helper_classes import lsf_status

import logging
logging.basicConfig(level=logging.INFO)

### Initialize the LSF manager

LSF manager is going to manage what jobs you have currently running, or created. It caches your information, so you can kill the notebook, turn-off your computer, go on vacation, and pick it up later. It will remember which jobs you haven't retrieved from the cluster, etc.

Note: it won't ask you for login unless you execute a method that requires remote access

In [ ]:
lsf=lsf_manager(user='your_username', host='your_cluster_hostname')

### Provide some smiles string(s)

and copy into them into ```my_smiles``` variable. It's a list of strings (a Python list can be of any size)

In [ ]:
my_smiles=[
    "C(C1C(C(C(C(O1)O)O)O)O)O",
    "c1ccc(F)cc1",
]

For a list of smiles, you can also read them out from a csv file in this way, assumed file name ```smiles_strings.csv```, also assuming the strings are in the first column, and there are no headers

In [ ]:
# my_smiles = pd.read_csv("smiles_strings.csv", header=None, usecols=[0])[0].tolist()
# my_smiles

### Initialize the molecules and generate conformations

We will use some ```for``` loops to loop over the smiles

In [ ]:
mols = []
for smile in my_smiles:
    mols.append(molecule(smile, max_num_conformers=3))

Draw some molecules

In [ ]:
mols[0].draw(2)

### Create gaussian jobs for each conformation

In [ ]:
for mol in mols:
    lsf.create_jobs_for_molecule(mol, workflow_type="equilibrium")

In [ ]:
lsf.get_job_stats(split_by_can=True)

### Submit the jobs to the server 

if you are not connected, it will ask for password + Duo

In [ ]:
lsf.remove_jobs(lsf.get_jobs())

In [ ]:
lsf.submit_jobs()

Check the status of the jobs, locally and remotely

In [ ]:
lsf.get_job_stats(split_by_can=True)

In [ ]:
lsf.bjobs(summary=False)

### Retrieve jobs

If there are some finished jobs the log files will be checked for completion and downloaded locally.

In [ ]:
lsf.retrieve_jobs()

### Resubmit incomplete jobs

In [ ]:
lsf.resubmit_incomplete_jobs()

### Upload molecules to the database

In [ ]:
sm.upload_done_molecules_to_db(tag="tutorial_AMZ")